In [15]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv('./Dane1.csv')

In [17]:
df.head()

,sezon,dzien_tygodnia,godzina,temperatura,pozycja_przed_meczem,rywal,frekwencja
0,2015/2016,niedziela,15:30,16,11,Nieciecza,10653
1,2015/2016,niedziela,15:30,26,7,Zagłębie Lubin,11055
2,2015/2016,niedziela,15:30,28,7,Ruch Chorzów,12758
3,2015/2016,niedziela,18:00,26,4,Legia Warszawa,22172
4,2015/2016,niedziela,18:00,15,10,Lech Poznań,19177


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   sezon                 105 non-null    object
 1   dzien_tygodnia        105 non-null    object
 2   godzina               105 non-null    object
 3   temperatura           105 non-null    int64 
 4   pozycja_przed_meczem  105 non-null    int64 
 5   rywal                 105 non-null    object
 6   frekwencja            105 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 5.9+ KB


In [19]:
correlation_matrix = df.corr()

C:\Users\jakub\AppData\Local\Temp\ipykernel_22836\4214245630.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


In [20]:
print(correlation_matrix)

                      temperatura  pozycja_przed_meczem  frekwencja
temperatura              1.000000              0.056218    0.511215
pozycja_przed_meczem     0.056218              1.000000   -0.332025
frekwencja               0.511215             -0.332025    1.000000


In [33]:
# Wylicz średnią frekwencję dla każdego rywala
mean_attendance_by_opponent = df.groupby('rywal')['frekwencja'].mean().sort_values().reset_index()

In [34]:
mean_attendance_by_opponent

,rywal,frekwencja
0,Z. Sosnowiec,6541.000000
1,Górnik Łęczna,7390.000000
2,Korona Kielce,7488.200000
3,Stal Mielec,7720.000000
4,Zagłębie Lubin,8217.666667
5,Piast Gliwice,8260.000000
6,Raków,8328.000000
7,P. Niepołomice,8478.000000
8,Warta Poznań,8481.000000
9,Podbeskidzie,8792.500000


In [35]:
df['rywal'].unique()

array(['Nieciecza', 'Zagłębie Lubin', 'Ruch Chorzów', 'Legia Warszawa',
       'Lech Poznań', 'Lechia Gdańsk', 'Wisła Kraków', 'Piast Gliwice',
       'Korona Kielce', 'Górnik Zabrze', 'Cracovia', 'Śląsk Wrocław',
       'Pogoń Szczecin', 'Górnik Łęczna', 'Podbeskidzie', 'Arka Gdynia',
       'Piast Gliwice\xa0', 'Wisła Płock', 'Sandecja', '\tWisła Kraków',
       'Miedź Legnica', 'Z. Sosnowiec', 'Widzew Łódź', 'Radomiak',
       'Stal Mielec', 'Legia Warszawa\xa0', 'Raków', 'Warta Poznań',
       'P. Niepołomice', 'ŁKS Łódź'], dtype=object)

In [36]:
df['rywal'] = df['rywal'].str.strip()

In [37]:
print(df['rywal'].unique())

['Nieciecza' 'Zagłębie Lubin' 'Ruch Chorzów' 'Legia Warszawa'
 'Lech Poznań' 'Lechia Gdańsk' 'Wisła Kraków' 'Piast Gliwice'
 'Korona Kielce' 'Górnik Zabrze' 'Cracovia' 'Śląsk Wrocław'
 'Pogoń Szczecin' 'Górnik Łęczna' 'Podbeskidzie' 'Arka Gdynia'
 'Wisła Płock' 'Sandecja' 'Miedź Legnica' 'Z. Sosnowiec' 'Widzew Łódź'
 'Radomiak' 'Stal Mielec' 'Raków' 'Warta Poznań' 'P. Niepołomice'
 'ŁKS Łódź']


In [51]:
mean_attendance_by_opponent = df.groupby('rywal')['frekwencja'].mean().sort_values().reset_index().round()

In [52]:
mean_attendance_by_opponent

,rywal,frekwencja
0,Z. Sosnowiec,6541.0
1,Górnik Łęczna,7390.0
2,Korona Kielce,7488.0
3,Stal Mielec,7720.0
4,Zagłębie Lubin,8218.0
5,Raków,8328.0
6,P. Niepołomice,8478.0
7,Warta Poznań,8481.0
8,Podbeskidzie,8792.0
9,Górnik Zabrze,8852.0


In [41]:
df['rywal'].is_unique

False

In [53]:
# Stwórz mapowanie dla kodowania rywali
opponent_encoding_map = {opponent: i+1 for i, opponent in enumerate(mean_attendance_by_opponent['rywal'])}

In [54]:
df['rywal_encoded'] = df['rywal'].map(opponent_encoding_map)

In [55]:
df

,sezon,dzien_tygodnia,godzina,temperatura,pozycja_przed_meczem,rywal,frekwencja,rywal_encoded
0,2015/2016,niedziela,15:30,16,11,Nieciecza,10653,16
1,2015/2016,niedziela,15:30,26,7,Zagłębie Lubin,11055,5
2,2015/2016,niedziela,15:30,28,7,Ruch Chorzów,12758,23
3,2015/2016,niedziela,18:00,26,4,Legia Warszawa,22172,27
4,2015/2016,niedziela,18:00,15,10,Lech Poznań,19177,25
...,...,...,...,...,...,...,...,...
100,2023/2024,sobota,15:00,11,1,Ruch Chorzów,11098,23
101,2023/2024,piątek,20:30,-2,3,Śląsk Wrocław,13840,14
102,2023/2024,sobota,15:00,22,1,ŁKS Łódź,15007,26
103,2023/2024,niedziela,15:00,16,1,Cracovia,18184,21


In [56]:
correlation_matrix = df.corr()

C:\Users\jakub\AppData\Local\Temp\ipykernel_22836\4214245630.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


In [57]:
print(correlation_matrix)

                      temperatura  pozycja_przed_meczem  frekwencja  \
temperatura              1.000000              0.056218    0.511215   
pozycja_przed_meczem     0.056218              1.000000   -0.332025   
frekwencja               0.511215             -0.332025    1.000000   
rywal_encoded            0.197674             -0.244795    0.624442   

                      rywal_encoded  
temperatura                0.197674  
pozycja_przed_meczem      -0.244795  
frekwencja                 0.624442  
rywal_encoded              1.000000  
